# Modelo de recomendacion

In [1]:
# importar librerías
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [2]:
df_model_recommend = pd.read_parquet("datasets/df_model_recommend.parquet")

In [3]:
df_model_recommend.dtypes

app_name     object
id          float64
tags_2       object
dtype: object

In [7]:
df_model_recommend['id'] = df_model_recommend['id'].astype(int)

In [6]:
df_model_recommend.head()
df_model_recommend["app_name"]

0                 Lost Summoner Kitty
1                           Ironbound
2             Real Pool 3D - Poolians
3               Battle Royale Trainer
4        SNOW - All Access Basic Pass
                     ...             
25535                    Kebab it Up!
25536                  Colony On Mars
25537        LOGistICAL: South Africa
25538                   Russian Roads
25539             EXIT 2 - Directions
Name: app_name, Length: 25540, dtype: object

In [6]:
# unir en cadenas separadas por espacios
df_model_recommend["tags_2"] = df_model_recommend["tags_2"].apply(lambda x: " ".join(x))

In [7]:
# pasar todo a minúsculas
df_model_recommend["tags_2"] = df_model_recommend["tags_2"].apply(lambda x: x.lower()) # sale un warning

Creamos un objeto CountVectorizer con las siguientes configuraciones:  
* max_features=5000: número máximo de características
* stop_words="english": palabras que se deben eliminar del texto.significativa.  

Que utilizaremos para convertir un conjunto de texto en un conjunto de vectores. Cada vector representa la  frecuencia de cada característica en el texto.

In [8]:
cv = CountVectorizer(max_features=5000, stop_words="english")

**Devolvemos el conjunto de vectores como una matriz.**

In [9]:
vectors = cv.fit_transform(df_model_recommend["tags_2"]).toarray()

**Creamos un objeto PorterStemmer que utilizaremos para reducir las palabras a su raíz.**

In [10]:
ps = PorterStemmer()

**Creamos una función que realizará estos pasos:**  
1. Divide el texto en palabras individuales.
2. Aplica el algoritmo de stemming de Porter a cada palabra individual.
3. Une las palabras individuales en una cadena de texto.

In [11]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

**Aplicamos la función a la columna tags_2**

In [12]:
df_model_recommend["tags_2"] = df_model_recommend["tags_2"].apply(stem)

**Calculamos la similitud coseno entre los vectores en la matriz**

In [13]:
similarity = cosine_similarity(vectors)

**Creamos la función recomendadora**

In [14]:
# función recomendadora
def recommend(game):
    game_index = df_model_recommend[df_model_recommend["app_name"] == game].index[0]
    distances = similarity[game_index]
    game_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    game_l = []
    for i in game_list:
        list_g = {df_model_recommend.iloc[i[0]].app_name}
        game_l.append(list_g)
        # print(i[0])
    return game_l

In [15]:
# prueba de la función
recommend("Half-Life: Blue Shift")

[{'Half-Life: Opposing Force'},
 {'Half-Life: Source'},
 {'Half-Life 2: Episode One'},
 {'Half-Life 2'},
 {'Unreal 2: The Awakening'}]